In [5]:
import os

In [7]:
os.system("git remote update");
os.system("git pull");

In [ ]:
def update(message):
    os.system('git commit -m "{}")

In [3]:
import numpy as np
import pandas as pd
#df = pd.read_csv("tmdb_5000_movies.csv")
df = pd.read_csv("IMDB-Movie-Data.csv")

# Dataset introduction and description

The IMDB Movie Dataset contains columns

1. A description of your data set.
Explain
• What the problem of interest is (i.e. what is your data about),
• Where you obtained the data,
• What has previously been done to the data. (i.e. if available go through
some of the original source papers and read what they did to the data
and summarize what were their results).
• What the primary machine learning modeling aim is for the data, i.e.
which attributes you feel are relevant when carrying out a classification,
a regression, a clustering, an association mining, and an anomaly detection
in the later reports and what you hope to accomplish using these
techniques. For instance, which attribute do you wish to explain in the
regression based on which other attributes? Which class label will you
02450 PROJECT 1 5 September, 2017
predict based on which other attributes in the classification task? If you
need to transform the data to admit these tasks, explain roughly how you
might do this (but don’t transform the data now!).


To get an overview of the dataset columns, we can look at the first entry in our dataframe.

In [4]:
df.head(1)
#dimensions = df.columns.values.tolist();

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0


2. A detailed explanation of the attributes of the data.
• Describe if the attributes are discrete/continous, Nominal/Ordinal/Interval/Ratio,
• give an account of whether there are data issues (i.e. missing values or
corrupted data) and describe them if so
• describe the basic summary statistics of the attributes.
If your data set contains many similar attributes, you may restrict yourself to
describing a few representative features (apply common sense).


3. Data visualization(s) based on suitable visualization techniques including
a principal component analysis (PCA).
Touch upon the following subjects, use visualizations when it appears sensible.
Keep in mind the ACCENT principles and Tufte’s guidelines when you visualize
the data.
• Are there issues with outliers in the data,
• do the attributes appear to be normal distributed,
• are variables correlated,
• does the primary machine learning modeling aim appear to be feasible
based on your visualizations.
There are three aspects that needs to be described when you carry out the
PCA analysis for the report:
• The amount of variation explained as a function of the number of PCA
components included,
• the principal directions of the considered PCA components (either find a
way to plot them or interpret them in terms of the features),
• the data projected onto the considered principal components.
If your attributes have very different scales it may be relevant to standardize
the data prior to the PCA analysis.

4. A discussion explaining what you have learned about the data.
Summarize here the most important things you have learned about the data
and give also your thoughts on whether your primary modeling task(s) appears
to be feasible based on your visualization.

## Descriptive Analysis

Analysis of relation between facebook likes and earnings

In [ ]:
work = df[['movie_facebook_likes','gross']].dropna();
work = work[(work != 0).all(1)];
work.plot.scatter('movie_facebook_likes', 'gross', grid=True);

Relation between movie_facebook_likes imdb_score

In [ ]:
work.nlargest(5, 'movie_facebook_likes')

In [ ]:
work = df[['imdb_score', 'movie_facebook_likes']].dropna();
work = work[(work != 0).all(1)];
work.plot.scatter('imdb_score', 'movie_facebook_likes', grid=True);

In [ ]:
dfc = df.corr()
np.fill_diagonal(dfc.values, 0)
dfc

In [ ]:
dfc.min()

In [ ]:
work = df[['imdb_score', 'movie_facebook_likes']].dropna();
work = work[(work != 0).all(1)];
work.plot.scatter('imdb_score', 'movie_facebook_likes', grid=True);